# Import

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from sqlalchemy import create_engine
import pandas as pd
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import datetime

In [7]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.style.use('fivethirtyeight')

# Calculate Status for all the patients that has responses

### Query data

In [8]:
df_patient_response = pd.read_csv('../../data/patient_response.csv')
df_patient = pd.read_csv('../../data/patient_detail.csv')
df_tmetryentry = pd.read_csv('../../data/tmetryentry.csv')

ImportError: No module named 'MySQLdb'

### How each dataframe looks like

In [ ]:
#Information for all of the entries
df_tmetryentry.head()

In [ ]:
df_tmetryentry[df_tmetryentry.strValue == '098d477fde14ee3ee30c9f363d5ac06f85c3ca51']

In [ ]:
#information on the answers
df_patient_response[df_patient_response.patient_id.isin([120])]

In [ ]:
#information on the patients
df_patient.head()

In [ ]:
df_patient[df_patient.hash_fin == '07e1b75fe53343221703226deb96b71bb6d90930']

In [ ]:
print(df_patient_response.columns, df_patient.columns)

In [ ]:
#matching patients with the answers
df1 = pd.merge(df_patient_response, df_patient[['id', 'hash_fin', 'date', 'checklist_id', 'unit_id',
                                                'visible_groups', 'los_icu', 'user_detail_id', 'just_date']],            
              
               left_on=['hash_fin','user_detail_id','unit_id','checklist_id'], 
               right_on=['hash_fin', 'user_detail_id','unit_id','checklist_id'])
# df_merge_patient_response[['checklist_id_x', 'question_id_x', 'workday_x', 'response', 'date_x', 'patient_id',]]
# df_merge_patient_response.columns

In [ ]:
#Please rename this, g1 has all the responses for each patient 
g = df1[df1.date_x >= df1.date_y].groupby(['hash_fin', 'id_x'])#user_detail_id', 'unit_id', 'checklist_id','workday'])#['date_x', 'visible_groups']
g1 = g.apply(lambda x: x.sort_values(by='date_y', ascending=False))
g1.drop_duplicates(subset='id_x', keep='first')

# Specific Categories

Yeah, so they want to look at three specific questions: 1) DVT, 2) Lines (specifically central lines if their version has it split into peripheral and central), 3) Foleys


[2:01] 
I think for each question, what they want to know is: for each week, 1) how many patients total were the questions answered, 2) for the patients where it was answered, what are the proportions of the different answers, 3) for patients where the 'bad' answer was given initially (no we can't discontinue the foley, no we can't pull the line, etc), how many days did they spend in the 'bad' answer category before switching to the 'good' category. [Taking into account where it was contraindicated]


[2:02] 
they want to see it because they measure all these things by another method (probably nursing reports) and they want to see if those two sources (us vs nursing reports) have any differences

# NSICU (unit 3)
01 ventilatorWeaning 

03 linesFood 

06 antibiotics 

07 dvt 

08 pressureUlcer 

10 NO 

### CTICU (unit 4)
01 ventilatorWeaning 

03 linesFood 

06 antibiotics 

07 dvt 

08 pressureUlcer 

10 NO 

In [73]:
checklist_query = "select * from backend_v10_checklist"
df_checklist = pd.read_sql(checklist_query, conn)
df_checklist

,id,public_id,name,text,in_production,unit_id
0,2,20,Test,"{""id"": ""20"", ""groups"": [{""id"": ""01"", ""dependen...",1,1
1,5,1,Initial checklist,"{""order"": ""(01,{03|06|07|08},10)"", ""id"": ""1"", ...",1,3
2,10,1,Initial checklist,"{""order"": ""(01,{03|06|07|08},10)"", ""id"": ""1"", ...",1,4
3,11,4,No ARDS with N/A (lowercase),"{""order"": ""(01,{03|06|07|08},10)"", ""groups"": [...",1,8
4,12,100,Leigh Wild Checklist,"{""order"": ""(01,{03|06|07|08},10)"", ""groups"": [...",1,8
5,13,2,Initial checklist with tracheostomy question,"{""description"": ""Initial checklist with trache...",1,3
6,14,2,Initial checklist with tracheostomy question,"{""description"": ""Initial checklist with trache...",1,4


In [94]:
import ast
unit3_2 =ast.literal_eval(df_checklist.text[2])
unit3_1=ast.literal_eval(df_checklist.text[6])

In [95]:
for key, val in unit3_1.items():
    print(key, val, type(val),len(val))
    print()

id 2 <class 'str'> 1

order (01,{03|06|07|08},10) <class 'str'> 21

groups [{'emrInfoId': 'ventilatorWeaning', 'sections': [{'id': '$0100', 'title': 'Sedation:', 'dependence': 'hasMedication', 'questions': [{'text': 'Has a sedation awakening trial (SAT) been performed today?', 'id': '$0101', 'dependence': 'true', 'indentation': 0, 'questionType': 'Q'}, {'text': 'Is it appropriate to perform an SAT today?', 'id': '$0102', 'dependence': "$0101=='N'", 'indentation': 0, 'questionType': 'A'}]}, {'id': '$0200', 'title': 'Ventilator weaning:', 'dependence': 'hasVentilation', 'questions': [{'text': 'Has a spontaneous breathing trial (SBT) been performed today?', 'id': '$0201', 'dependence': 'true', 'indentation': 0, 'questionType': 'Q'}, {'text': 'Can the patient be extubated today?', 'id': '$0202', 'dependence': "$0201=='Y'", 'indentation': 0, 'questionType': 'Q'}, {'text': 'Does the patient have a tracheostomy or is scheduled for tracheostomy placement?', 'id': '$0205', 'dependence': "$0201=

In [96]:
for a in unit3_1['groups']:
    print(a['id'], a['emrInfoId'], '\n')

01 ventilatorWeaning 

03 linesFood 

06 antibiotics 

07 dvt 

08 pressureUlcer 

10 NO 



In [97]:
for b in unit3_2['groups']:
    print(b['id'], b['emrInfoId'], '\n')

01 ventilatorWeaning 

03 linesFood 

06 antibiotics 

07 dvt 

08 pressureUlcer 

10 NO 

